### Random Forest Model for Twitter Bot Account Detection

## Import libraries

In [243]:
import pandas as pd
import numpy as mp
import seaborn as sns

## Load Dataset

In [244]:
df = pd.read_csv('dataset.csv')
df.head()

,user_id,screen_name,is_bot,account_age,is_blue_verified,is_verified,profile_description,follwing_count,followers_count,following_to_followers_ratio,...,freq_of_tweets,identical_tweets_count_ratio,avg_replies_per_tweet,avg_urls_per_tweet,avg_likes_per_tweet,possibly_sensitive,profile_image_url,profile_banner_url,is_profile_image_valid,followers_to_likes_ratio
0,2718436417,DJGruuvan,0,10.256,0,0,"THE DJ GRUUVAN! You know who it is, baby.",108,96,1.1250,...,3.7051,0,0.1333,0.0667,0.2667,0,https://pbs.twimg.com/profile_images/497928954...,NaN,0,360.0000
1,792615528791703553,Cc54518746C,0,8.030,0,0,NaN,93,24,3.8750,...,771.6065,0,0.0000,0.9286,0.0952,0,https://abs.twimg.com/sticky/default_profile_i...,NaN,0,252.0000
2,3287012484,penelope20mn,0,9.306,0,0,Racial Justice & Equity · LGBTQIA+ · Pro-Choic...,4078,12122,0.3364,...,8622.9314,0,0.0000,0.0000,0.0000,0,https://pbs.twimg.com/profile_images/755098445...,https://pbs.twimg.com/profile_banners/32870124...,0,0.0000
3,4462881555,ShitpostBot5000,1,8.912,0,0,randomly generated shitposting every 30 minute...,11,267426,0.0000,...,15932.4506,0,1.0900,0.0000,41.7300,1,https://pbs.twimg.com/profile_images/765508475...,https://pbs.twimg.com/profile_banners/44628815...,0,6408.4831
4,804681,SethNickerson,0,17.695,0,0,Proud. Sane. Freedom loving. Idaho Native. Rad...,354,384,0.9219,...,164.9053,0,0.0725,0.1449,0.8841,0,https://pbs.twimg.com/profile_images/185438232...,https://pbs.twimg.com/profile_banners/804681/1...,0,434.3607


In [245]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         15 non-null     int64  
 1   screen_name                     15 non-null     object 
 2   is_bot                          15 non-null     int64  
 3   account_age                     15 non-null     float64
 4   is_blue_verified                15 non-null     int64  
 5   is_verified                     15 non-null     int64  
 6   profile_description             13 non-null     object 
 7   follwing_count                  15 non-null     int64  
 8   followers_count                 15 non-null     int64  
 9   following_to_followers_ratio    15 non-null     float64
 10  fast_followers_count            15 non-null     int64  
 11  normal_followers_count          15 non-null     int64  
 12  fast_to_normal_followers_ratio  15 non

In [246]:
df.isnull().sum()

user_id                           0
screen_name                       0
is_bot                            0
account_age                       0
is_blue_verified                  0
is_verified                       0
profile_description               2
follwing_count                    0
followers_count                   0
following_to_followers_ratio      0
fast_followers_count              0
normal_followers_count            0
fast_to_normal_followers_ratio    0
tweets_count                      0
parsed_tweets_count               0
parsed_retweets_count             0
replies_ratio                     0
retweets_ratio                    0
freq_of_tweets                    0
identical_tweets_count_ratio      0
avg_replies_per_tweet             0
avg_urls_per_tweet                0
avg_likes_per_tweet               0
possibly_sensitive                0
profile_image_url                 0
profile_banner_url                3
is_profile_image_valid            0
followers_to_likes_ratio    

## Replace any null values with empty string

In [247]:
df.fillna('', inplace=True)

In [248]:
df.isnull().sum()

user_id                           0
screen_name                       0
is_bot                            0
account_age                       0
is_blue_verified                  0
is_verified                       0
profile_description               0
follwing_count                    0
followers_count                   0
following_to_followers_ratio      0
fast_followers_count              0
normal_followers_count            0
fast_to_normal_followers_ratio    0
tweets_count                      0
parsed_tweets_count               0
parsed_retweets_count             0
replies_ratio                     0
retweets_ratio                    0
freq_of_tweets                    0
identical_tweets_count_ratio      0
avg_replies_per_tweet             0
avg_urls_per_tweet                0
avg_likes_per_tweet               0
possibly_sensitive                0
profile_image_url                 0
profile_banner_url                0
is_profile_image_valid            0
followers_to_likes_ratio    

In [249]:
df.head()

,user_id,screen_name,is_bot,account_age,is_blue_verified,is_verified,profile_description,follwing_count,followers_count,following_to_followers_ratio,...,freq_of_tweets,identical_tweets_count_ratio,avg_replies_per_tweet,avg_urls_per_tweet,avg_likes_per_tweet,possibly_sensitive,profile_image_url,profile_banner_url,is_profile_image_valid,followers_to_likes_ratio
0,2718436417,DJGruuvan,0,10.256,0,0,"THE DJ GRUUVAN! You know who it is, baby.",108,96,1.1250,...,3.7051,0,0.1333,0.0667,0.2667,0,https://pbs.twimg.com/profile_images/497928954...,,0,360.0000
1,792615528791703553,Cc54518746C,0,8.030,0,0,,93,24,3.8750,...,771.6065,0,0.0000,0.9286,0.0952,0,https://abs.twimg.com/sticky/default_profile_i...,,0,252.0000
2,3287012484,penelope20mn,0,9.306,0,0,Racial Justice & Equity · LGBTQIA+ · Pro-Choic...,4078,12122,0.3364,...,8622.9314,0,0.0000,0.0000,0.0000,0,https://pbs.twimg.com/profile_images/755098445...,https://pbs.twimg.com/profile_banners/32870124...,0,0.0000
3,4462881555,ShitpostBot5000,1,8.912,0,0,randomly generated shitposting every 30 minute...,11,267426,0.0000,...,15932.4506,0,1.0900,0.0000,41.7300,1,https://pbs.twimg.com/profile_images/765508475...,https://pbs.twimg.com/profile_banners/44628815...,0,6408.4831
4,804681,SethNickerson,0,17.695,0,0,Proud. Sane. Freedom loving. Idaho Native. Rad...,354,384,0.9219,...,164.9053,0,0.0725,0.1449,0.8841,0,https://pbs.twimg.com/profile_images/185438232...,https://pbs.twimg.com/profile_banners/804681/1...,0,434.3607


## Feature Engineering

### Drop irrelevent values to training

In [250]:
df.drop(columns=['user_id', 'screen_name', 'profile_description', 'profile_image_url', 'profile_banner_url'], inplace=True)

In [251]:
df.head()

,is_bot,account_age,is_blue_verified,is_verified,follwing_count,followers_count,following_to_followers_ratio,fast_followers_count,normal_followers_count,fast_to_normal_followers_ratio,...,replies_ratio,retweets_ratio,freq_of_tweets,identical_tweets_count_ratio,avg_replies_per_tweet,avg_urls_per_tweet,avg_likes_per_tweet,possibly_sensitive,is_profile_image_valid,followers_to_likes_ratio
0,0,10.256,0,0,108,96,1.1250,0,96,0.0,...,0.0,0.0625,3.7051,0,0.1333,0.0667,0.2667,0,0,360.0000
1,0,8.030,0,0,93,24,3.8750,0,24,0.0,...,0.0,0.6182,771.6065,0,0.0000,0.9286,0.0952,0,0,252.0000
2,0,9.306,0,0,4078,12122,0.3364,0,12122,0.0,...,0.0,0.0000,8622.9314,0,0.0000,0.0000,0.0000,0,0,0.0000
3,1,8.912,0,0,11,267426,0.0000,0,267426,0.0,...,0.0,0.0000,15932.4506,0,1.0900,0.0000,41.7300,1,0,6408.4831
4,0,17.695,0,0,354,384,0.9219,0,384,0.0,...,0.0,0.4103,164.9053,0,0.0725,0.1449,0.8841,0,0,434.3607


## Create target variable

In [252]:
Y = df.is_bot
Y.head()

0    0
1    0
2    0
3    1
4    0
Name: is_bot, dtype: int64

## Drop the target variable : is_bot

In [253]:
df.drop('is_bot', inplace=True, axis=1)

In [254]:
df.head()

,account_age,is_blue_verified,is_verified,follwing_count,followers_count,following_to_followers_ratio,fast_followers_count,normal_followers_count,fast_to_normal_followers_ratio,tweets_count,...,replies_ratio,retweets_ratio,freq_of_tweets,identical_tweets_count_ratio,avg_replies_per_tweet,avg_urls_per_tweet,avg_likes_per_tweet,possibly_sensitive,is_profile_image_valid,followers_to_likes_ratio
0,10.256,0,0,108,96,1.1250,0,96,0.0,38,...,0.0,0.0625,3.7051,0,0.1333,0.0667,0.2667,0,0,360.0000
1,8.030,0,0,93,24,3.8750,0,24,0.0,6196,...,0.0,0.6182,771.6065,0,0.0000,0.9286,0.0952,0,0,252.0000
2,9.306,0,0,4078,12122,0.3364,0,12122,0.0,80245,...,0.0,0.0000,8622.9314,0,0.0000,0.0000,0.0000,0,0,0.0000
3,8.912,0,0,11,267426,0.0000,0,267426,0.0,141990,...,0.0,0.0000,15932.4506,0,1.0900,0.0000,41.7300,1,0,6408.4831
4,17.695,0,0,354,384,0.9219,0,384,0.0,2918,...,0.0,0.4103,164.9053,0,0.0725,0.1449,0.8841,0,0,434.3607


In [255]:
X = df

## Split Data into Test & Training

In [256]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

30% for testing, 70% for training, use random state to produce same outcomes

In [257]:
print('X_train: ', X_train.shape)
print('X_test: ', X_test.shape)
print('Y_train: ', Y_train.shape)
print('Y_test: ', Y_test.shape)

X_train:  (10, 22)
X_test:  (5, 22)
Y_train:  (10,)
Y_test:  (5,)


## Training Random Forest Classification on Training Set

In [258]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 7, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=7, random_state=0)

## Predicting Test Results

In [259]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, 0, 0])

## Confusion Matrix

In [260]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [261]:
cm = confusion_matrix(Y_test, y_pred)
print(cm)

[[4 0]
 [1 0]]


In [262]:
accuracy_score(Y_test, y_pred)

0.8

In [263]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       0.00      0.00      0.00         1

    accuracy                           0.80         5
   macro avg       0.40      0.50      0.44         5
weighted avg       0.64      0.80      0.71         5



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

poor precision and accuracy due to small dataset. will increase with larger dataset.